# Jaguar Re-ID

## Score: .771

In [ ]:
import os
import math
import random
from pathlib import Path
from collections import defaultdict

import numpy as np
import pandas as pd
from PIL import Image
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Sampler

import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

In [ ]:
# =============================================================================
# CONFIG
# =============================================================================
INPUT_DIR = Path('/kaggle/input/jaguar-re-identification-challenge')
DATA_DIR = None
if (INPUT_DIR / 'train.csv').exists():
    DATA_DIR = INPUT_DIR
if DATA_DIR is None and Path('/kaggle/input').exists():
    for d in Path('/kaggle/input').iterdir():
        if not d.is_dir() or 'convnext' in d.name.lower():
            continue
        if (d / 'train.csv').exists() and (d / 'test.csv').exists():
            DATA_DIR = d
            break
        for sub in d.iterdir():
            if sub.is_dir() and (sub / 'train.csv').exists() and (sub / 'test.csv').exists():
                DATA_DIR = sub
                break
        if DATA_DIR is not None:
            break
INPUT_DIR = DATA_DIR if DATA_DIR is not None else INPUT_DIR
WORKING_DIR = Path('/kaggle/working')

MODEL_INPUT_DIR = Path('/kaggle/input/eva02-default-v1/models_eva02_large_single_run')
if not MODEL_INPUT_DIR.exists() and Path('/kaggle/input').exists():
    for d in Path('/kaggle/input').iterdir():
        if not d.is_dir():
            continue
        sub = d / 'models_eva02_large_single_run'
        if sub.is_dir():
            MODEL_INPUT_DIR = sub
            break
        if any(d.rglob('*.safetensors')) or any(d.rglob('*.pth')) or any(d.rglob('*.pt')) or any(d.rglob('*.bin')):
            MODEL_INPUT_DIR = d
            break
    else:
        MODEL_INPUT_DIR = Path('/kaggle/input/eva02-default-v1')

class CFG:
    data_dir = INPUT_DIR
    train_csv = data_dir / 'train.csv'
    test_csv = data_dir / 'test.csv'
    train_dir = data_dir / 'train' / 'train'
    test_dir = data_dir / 'test' / 'test'
    out_dir = WORKING_DIR
    model_input_dir = MODEL_INPUT_DIR
    
    backbone = 'eva02_large_patch14_clip_336.merged2b'
    image_size = 448
    num_classes = 31

    epochs = 6
    batch_size = 4
    grad_accum_steps = 4
    lr = 2e-5
    weight_decay = 1e-3
    arcface_s = 30.0
    arcface_m = 0.5
    label_smoothing = 0.05
    samples_per_class = 60
    early_stop_patience = 2

    use_tta = True
    use_qe = True
    qe_top_k = 3
    use_rerank = True
    rerank_lambda = 0.3
    train_seeds = [42]

    do_pl = True
    pl_threshold = 0.90
    pl_max_add = 500
    pl_epochs = 5
    pl_lr = 1e-5
    pl_batch_size = 2
    pl_grad_accum = 4

    num_workers = 0
    mixed_precision = True

In [ ]:
# =============================================================================
# DATA
# =============================================================================
full_train = pd.read_csv(CFG.train_csv)
test_df = pd.read_csv(CFG.test_csv)
val_indices = []
for gt, grp in full_train.groupby('ground_truth'):
    idx = grp.index.tolist()
    if len(idx) >= 2:
        val_indices.extend(random.sample(idx, 2))
train_df = full_train.drop(index=val_indices).reset_index(drop=True)
val_df = full_train.loc[val_indices].reset_index(drop=True)
print(f"Train: {len(train_df)} | Val: {len(val_df)} | Test pairs: {len(test_df)}")

In [ ]:
# =============================================================================
# TRANSFORMS
# =============================================================================
NORM_MEAN, NORM_STD = ((0.481, 0.457, 0.408), (0.268, 0.261, 0.275)) if 'eva' in CFG.backbone else ((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

def get_train_transforms():
    return A.Compose([
        A.LongestMaxSize(max_size=CFG.image_size),
        A.PadIfNeeded(CFG.image_size, CFG.image_size, border_mode=0),
        A.HorizontalFlip(p=0.5),
        A.Affine(scale=(0.9, 1.1), rotate=(-12, 12), shear=(-8, 8), p=0.5),
        A.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1, p=0.6),
        A.CoarseDropout(num_holes_range=(4, 12), hole_height_range=(16, 48), hole_width_range=(16, 48), p=0.3),
        A.Normalize(mean=NORM_MEAN, std=NORM_STD),
        ToTensorV2(),
    ])

def get_test_transforms(flip=False, size=None):
    sz = size if size is not None else CFG.image_size
    t = [
        A.LongestMaxSize(max_size=sz),
        A.PadIfNeeded(sz, sz, border_mode=0),
    ]
    if sz != CFG.image_size:
        t.append(A.Resize(CFG.image_size, CFG.image_size))
    if flip:
        t.append(A.HorizontalFlip(p=1.0))
    t.extend([A.Normalize(mean=NORM_MEAN, std=NORM_STD), ToTensorV2()])
    return A.Compose(t)

In [ ]:
# =============================================================================
# DATASET & SAMPLER
# =============================================================================
class JaguarDataset(Dataset):
    def __init__(self, df, img_dir, transform, label_map=None, use_dir_col=False):
        self.df = df.reset_index(drop=True)
        self.img_dir = Path(img_dir)
        self.transform = transform
        self.use_dir_col = use_dir_col
        if label_map is not None:
            self.label_map = label_map
        else:
            unique_ids = sorted(df['ground_truth'].unique())
            self.label_map = {name: i for i, name in enumerate(unique_ids)}
        self.labels = [self.label_map[gt] for gt in df['ground_truth']]
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = Path(row['dir']) / row['filename'] if self.use_dir_col and 'dir' in row else self.img_dir / row['filename']
        img = np.array(Image.open(img_path).convert('RGB'))
        img = self.transform(image=img)['image']
        return img, torch.tensor(self.labels[idx], dtype=torch.long)

class JaguarTestDataset(Dataset):
    def __init__(self, filenames, img_dir, transform):
        self.filenames = filenames
        self.img_dir = Path(img_dir)
        self.transform = transform
    def __len__(self):
        return len(self.filenames)
    def __getitem__(self, idx):
        fname = self.filenames[idx]
        img = np.array(Image.open(self.img_dir / fname).convert('RGB'))
        img = self.transform(image=img)['image']
        return img, fname

class BalancedSampler(Sampler):
    def __init__(self, labels, samples_per_class):
        self.labels = labels
        self.samples_per_class = samples_per_class
        self.class_indices = defaultdict(list)
        for idx, label in enumerate(labels):
            self.class_indices[label].append(idx)
        self.num_classes = len(self.class_indices)
    def __iter__(self):
        indices = []
        for label in self.class_indices:
            class_idx = self.class_indices[label]
            sampled = random.sample(class_idx, self.samples_per_class) if len(class_idx) >= self.samples_per_class else random.choices(class_idx, k=self.samples_per_class)
            indices.extend(sampled)
        random.shuffle(indices)
        return iter(indices)
    def __len__(self):
        return self.num_classes * self.samples_per_class

In [ ]:
# =============================================================================
# MODEL
# =============================================================================
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super().__init__()
        self.p = nn.Parameter(torch.ones(1) * p)
        self.eps = eps
    def forward(self, x):
        return F.avg_pool2d(x.clamp(min=self.eps).pow(self.p), (x.size(-2), x.size(-1))).pow(1.0 / self.p)

class ArcFaceLoss(nn.Module):
    def __init__(self, in_features, num_classes, s=30.0, m=0.5):
        super().__init__()
        self.s, self.m = s, m
        self.weight = nn.Parameter(torch.FloatTensor(num_classes, in_features))
        nn.init.xavier_uniform_(self.weight)
    def forward(self, x, labels):
        cosine = F.linear(F.normalize(x), F.normalize(self.weight))
        theta = torch.acos(cosine.clamp(-1 + 1e-7, 1 - 1e-7))
        target_logits = torch.cos(theta + self.m)
        one_hot = F.one_hot(labels, num_classes=cosine.size(1)).float()
        output = cosine * (1 - one_hot) + target_logits * one_hot
        return output * self.s

class JaguarModel(nn.Module):
    def __init__(self):
        super().__init__()
        model_dir = getattr(CFG, 'model_input_dir', None)
        use_kaggle_weights = model_dir and Path(model_dir).exists()
        if not use_kaggle_weights:
            raise FileNotFoundError('Internet off: add a Kaggle dataset with backbone weights and set CFG.model_input_dir to its path.')
        kwargs = {'pretrained': False, 'num_classes': 0}
        if 'vit' in CFG.backbone or 'eva' in CFG.backbone:
            kwargs['img_size'] = CFG.image_size
        self.backbone = timm.create_model(CFG.backbone, **kwargs)
        self._load_backbone_from_kaggle(Path(model_dir))
        self.feat_dim = self.backbone.num_features
        self.gem = GeM()
        self.bn = nn.BatchNorm1d(self.feat_dim)
        self.dropout = nn.Dropout(0.1)
        self.arcface = ArcFaceLoss(self.feat_dim, CFG.num_classes, CFG.arcface_s, CFG.arcface_m)
        print(f"Loaded {CFG.backbone} | Features: {self.feat_dim} (from Kaggle input)")
    def _load_backbone_from_kaggle(self, model_dir):
        model_dir = Path(model_dir)
        candidates = list(model_dir.rglob('*.safetensors')) + list(model_dir.rglob('*.pth')) + list(model_dir.rglob('*.pt')) + list(model_dir.rglob('*.bin')) + list(model_dir.glob('*.safetensors')) + list(model_dir.glob('*.pth')) + list(model_dir.glob('*.pt')) + list(model_dir.glob('*.bin'))
        if not candidates:
            raise FileNotFoundError(f'No .safetensors/.pth/.pt/.bin in {model_dir}. Add a dataset with EVA-02 Large weights.')
        path = candidates[0]
        if path.suffix == '.safetensors':
            try:
                from safetensors.torch import load_file
                ckpt = load_file(str(path))
            except ImportError:
                raise ImportError('Install safetensors: pip install safetensors')
        else:
            ckpt = torch.load(path, map_location='cpu')
        if isinstance(ckpt, dict):
            for key in ('state_dict', 'model', 'module'):
                if key in ckpt and isinstance(ckpt[key], dict):
                    ckpt = ckpt[key]
                    break
        state = self.backbone.state_dict()
        def _map_key(k):
            for p in ('module.', 'backbone.', 'model.', 'encoder.'):
                if k.startswith(p): k = k[len(p):]
            return k
        loaded = {_map_key(k): v for k, v in ckpt.items()}
        matched = {k: v for k, v in loaded.items() if k in state and state[k].shape == v.shape}
        self.backbone.load_state_dict(matched, strict=False)
        print(f"Backbone: loaded {len(matched)}/{len(state)} params from {path.name}")
    def extract(self, x):
        features = self.backbone.forward_features(x)
        if features.dim() == 3:
            B, N, C = features.shape
            H = W = int(math.sqrt(N))
            if H * W != N:
                features = features[:, -H * W:, :]
            features = features.permute(0, 2, 1).reshape(B, C, H, W)
            emb = self.gem(features).flatten(1)
        else:
            emb = self.gem(features).flatten(1)
        emb = self.bn(emb)
        return emb
    def forward(self, x, labels=None):
        emb = self.extract(x)
        if labels is not None:
            emb = self.dropout(emb)
            return self.arcface(emb, labels)
        return emb

In [ ]:
# =============================================================================
# POST-PROCESSING
# =============================================================================
def query_expansion(emb, top_k=None, verbose=True):
    top_k = top_k or getattr(CFG, 'qe_top_k', 3)
    if verbose:
        print("Applying Query Expansion...")
    sims = emb @ emb.T
    indices = np.argsort(-sims, axis=1)[:, :top_k]
    new_emb = np.array([np.mean(emb[indices[i]], axis=0) for i in range(len(emb))], dtype=emb.dtype)
    return new_emb / (np.linalg.norm(new_emb, axis=1, keepdims=True) + 1e-8)

def k_reciprocal_rerank(prob, k1=20, lambda_value=None, verbose=True):
    lambda_value = lambda_value or getattr(CFG, 'rerank_lambda', 0.3)
    if verbose:
        print("Applying Re-ranking...")
    q_g_dist = 1 - prob
    original_dist = q_g_dist.copy()
    initial_rank = np.argsort(original_dist, axis=1)
    nn_k1 = []
    for i in range(prob.shape[0]):
        forward_k1 = initial_rank[i, :k1+1]
        backward_k1 = initial_rank[forward_k1, :k1+1]
        fi = np.where(backward_k1 == i)[0]
        nn_k1.append(forward_k1[fi])
    jaccard_dist = np.zeros_like(original_dist)
    for i in range(prob.shape[0]):
        ind_non_zero = np.where(original_dist[i, :] < 0.6)[0]
        for j in ind_non_zero:
            if len(np.intersect1d(nn_k1[i], nn_k1[j])) > 0:
                inter = len(np.intersect1d(nn_k1[i], nn_k1[j]))
                union = len(np.union1d(nn_k1[i], nn_k1[j]))
                jaccard_dist[i, j] = 1 - inter / union
    return 1 - (jaccard_dist * lambda_value + original_dist * (1 - lambda_value))

In [ ]:
# =============================================================================
# VALIDATION
# =============================================================================
def compute_val_mAP(emb, labels):
    emb = np.asarray(emb)
    labels = np.asarray(labels)
    n = len(labels)
    sim = emb @ emb.T
    mAP_per_id = []
    for c in np.unique(labels):
        idx = np.where(labels == c)[0]
        if len(idx) < 2:
            continue
        aps = []
        for q in idx:
            gallery = np.array([i for i in range(n) if i != q])
            rel = (labels[gallery] == c).astype(float)
            if rel.sum() == 0:
                continue
            order = np.argsort(-sim[q, gallery])
            rel_ord = rel[order]
            prec = np.cumsum(rel_ord) / (1 + np.arange(len(rel_ord)))
            ap = (prec[rel_ord == 1].sum()) / rel.sum()
            aps.append(ap)
        if aps:
            mAP_per_id.append(np.mean(aps))
    return float(np.mean(mAP_per_id)) if mAP_per_id else 0.0

In [ ]:
# =============================================================================
# TRAINING
# =============================================================================
train_dataset = JaguarDataset(train_df, CFG.train_dir, get_train_transforms())
val_dataset = JaguarDataset(val_df, CFG.train_dir, get_test_transforms(flip=False))
grad_accum = CFG.grad_accum_steps

for seed in CFG.train_seeds:
    seed_everything(seed)
    train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, sampler=BalancedSampler(train_dataset.labels, CFG.samples_per_class), num_workers=CFG.num_workers, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=CFG.num_workers)
    model = JaguarModel().to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CFG.epochs)
    scaler = torch.amp.GradScaler('cuda')
    criterion = nn.CrossEntropyLoss(label_smoothing=CFG.label_smoothing)
    best_val_mAP = 0.0
    no_improve = 0
    print(f"--- Seed {seed} ---")
    for epoch in range(CFG.epochs):
        model.train()
        total_loss = 0
        optimizer.zero_grad()
        pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{CFG.epochs}')
        for step, (imgs, labels) in enumerate(pbar):
            if epoch == 0:
                for pg in optimizer.param_groups:
                    pg['lr'] = CFG.lr * (0.1 + 0.9 * (step + 1) / len(train_loader))
            imgs, labels = imgs.to(device), labels.to(device)
            with torch.amp.autocast('cuda'):
                logits = model(imgs, labels)
                loss = criterion(logits, labels) / grad_accum
            scaler.scale(loss).backward()
            if (step + 1) % grad_accum == 0 or (step + 1) == len(train_loader):
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
            total_loss += loss.item() * grad_accum
            pbar.set_postfix({'loss': f'{loss.item() * grad_accum:.4f}'})
        avg_loss = total_loss / len(train_loader)
        model.eval()
        emb_list, label_list = [], []
        with torch.no_grad():
            for imgs, labels in tqdm(val_loader, desc='Val', leave=False):
                emb = model(imgs.to(device))
                emb_list.append(F.normalize(emb, dim=1).cpu().numpy())
                label_list.append(labels.numpy())
        emb_val = np.concatenate(emb_list)
        labels_val = np.concatenate(label_list)
        val_mAP = compute_val_mAP(emb_val, labels_val)
        scheduler.step()
        print(f"Epoch {epoch+1} | Loss: {avg_loss:.4f} | Val mAP: {val_mAP:.4f} | LR: {scheduler.get_last_lr()[0]:.2e}")
        if val_mAP > best_val_mAP:
            best_val_mAP = val_mAP
            no_improve = 0
            torch.save(model.state_dict(), CFG.out_dir / f'best_model_seed{seed}.pth')
        else:
            no_improve += 1
            if no_improve >= CFG.early_stop_patience:
                print(f"Early stop at epoch {epoch+1}")
                break
    print(f"Seed {seed} done | Best val mAP: {best_val_mAP:.4f}")
print("Training complete.")

In [ ]:
# =============================================================================
# INFERENCE
# =============================================================================
unique_images = sorted(set(test_df['query_image']) | set(test_df['gallery_image']))
print(f"Extracting embeddings for {len(unique_images)} images...")

def extract_embeddings(transform, m):
    loader = DataLoader(JaguarTestDataset(unique_images, CFG.test_dir, transform), batch_size=CFG.batch_size, shuffle=False, num_workers=CFG.num_workers)
    feats, names = [], []
    with torch.no_grad():
        for imgs, fnames in tqdm(loader, leave=False):
            emb = m(imgs.to(device))
            feats.append(F.normalize(emb, dim=1).cpu())
            names.extend(fnames)
    return torch.cat(feats, dim=0), names

def get_embeddings(m):
    e1, names = extract_embeddings(get_test_transforms(flip=False), m)
    if CFG.use_tta:
        e2, _ = extract_embeddings(get_test_transforms(flip=True), m)
        emb = F.normalize((e1 + e2) / 2, dim=1)
    else:
        emb = e1
    return emb.numpy(), names

pl_model = None
if getattr(CFG, 'do_pl', False):
    def _gen_pl(model, test_fnames, lm, inv_lm, thresh, max_add):
        model.eval()
        loader = DataLoader(JaguarTestDataset(test_fnames, CFG.test_dir, get_test_transforms(flip=False)), batch_size=CFG.batch_size, shuffle=False, num_workers=CFG.num_workers)
        centroids = F.normalize(model.arcface.weight.data, dim=1)
        out = []
        with torch.no_grad():
            for imgs, fnames in tqdm(loader, desc='PL'):
                emb = F.normalize(model(imgs.to(device)), dim=1)
                probs = F.softmax(emb @ centroids.T.cuda() * CFG.arcface_s, dim=1)
                mp, pi = probs.max(1)
                for i, fn in enumerate(fnames):
                    if mp[i].item() >= thresh:
                        out.append({'filename': fn, 'ground_truth': inv_lm[pi[i].item()], 'dir': str(CFG.test_dir)})
        pl_df = pd.DataFrame(out)
        return pl_df.sample(n=min(len(pl_df), max_add), random_state=42) if len(pl_df) > max_add else pl_df
    inv_lm = {v: k for k, v in train_dataset.label_map.items()}
    tf = sorted(set(test_df['query_image']) | set(test_df['gallery_image']))
    pl_dfs = []
    for seed in CFG.train_seeds:
        ckpt = CFG.out_dir / f'best_model_seed{seed}.pth'
        if ckpt.exists():
            m = JaguarModel().to(device)
            m.load_state_dict(torch.load(ckpt))
            pl_dfs.append(_gen_pl(m, tf, train_dataset.label_map, inv_lm, CFG.pl_threshold, CFG.pl_max_add))
            del m
            torch.cuda.empty_cache()
    if pl_dfs:
        pl_df = pd.concat(pl_dfs).drop_duplicates(subset=['filename'])
        tr = train_df.copy()
        tr['dir'] = str(CFG.train_dir)
        comb = pd.concat([tr, pl_df], ignore_index=True)
        ds = JaguarDataset(comb, CFG.train_dir, get_train_transforms(), label_map=train_dataset.label_map, use_dir_col=True)
        pl_bs = getattr(CFG, 'pl_batch_size', 2)
        pl_ga = getattr(CFG, 'pl_grad_accum', 4)
        ld = DataLoader(ds, batch_size=pl_bs, sampler=BalancedSampler(ds.labels, CFG.samples_per_class), num_workers=CFG.num_workers, pin_memory=True)
        pl_model = JaguarModel().to(device)
        pl_model.load_state_dict(torch.load(CFG.out_dir / f'best_model_seed{CFG.train_seeds[0]}.pth'))
        opt = torch.optim.AdamW(pl_model.parameters(), lr=CFG.pl_lr, weight_decay=CFG.weight_decay)
        sch = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=CFG.pl_epochs)
        scal = torch.amp.GradScaler('cuda')
        print(f"PL: {len(pl_df)} samples | FT {CFG.pl_epochs} ep")
        for ep in range(CFG.pl_epochs):
            pl_model.train()
            opt.zero_grad()
            for step, (imgs, labels) in enumerate(tqdm(ld, desc=f'PL ep {ep+1}')):
                imgs, labels = imgs.to(device), labels.to(device)
                with torch.amp.autocast('cuda'):
                    loss = criterion(pl_model(imgs, labels), labels) / pl_ga
                scal.scale(loss).backward()
                if (step + 1) % pl_ga == 0 or (step + 1) == len(ld):
                    scal.step(opt)
                    scal.update()
                    opt.zero_grad()
            sch.step()
        torch.save(pl_model.state_dict(), CFG.out_dir / 'pl_model.pth')
        print('PL done.')
if pl_model is not None:
    pl_model.eval()
    emb, names = get_embeddings(pl_model)
    emb = emb.astype(np.float32) / (np.linalg.norm(emb, axis=1, keepdims=True) + 1e-8)
    print('Using PL model.')
else:
    emb_list = []
    for seed in CFG.train_seeds:
        ckpt_path = CFG.out_dir / f'best_model_seed{seed}.pth'
        if not ckpt_path.exists():
            raise FileNotFoundError(f"Missing {ckpt_path}")
        model = JaguarModel().to(device)
        model.load_state_dict(torch.load(ckpt_path))
        model.eval()
        e, names = get_embeddings(model)
        emb_list.append(e)
    emb = np.mean(emb_list, axis=0).astype(np.float32)
    emb = emb / (np.linalg.norm(emb, axis=1, keepdims=True) + 1e-8)
img_map = {n: i for i, n in enumerate(names)}

In [ ]:
# PL runs inside INFERENCE cell above.

In [ ]:
# =============================================================================
# SUBMISSION
# =============================================================================
emb_raw = emb.copy()
e = query_expansion(emb_raw.copy(), top_k=CFG.qe_top_k, verbose=True) if CFG.use_qe else emb_raw
sim = e @ e.T
if CFG.use_rerank:
    sim = k_reciprocal_rerank(sim, k1=20, lambda_value=CFG.rerank_lambda, verbose=True)
preds = [float(np.clip(sim[img_map[row['query_image']], img_map[row['gallery_image']]], 0, 1)) for _, row in test_df.iterrows()]
path_sub = CFG.out_dir / 'submission.csv'
pd.DataFrame({'row_id': test_df['row_id'], 'similarity': preds}).to_csv(path_sub, index=False)
print(f"Saved: {path_sub} | mean similarity: {np.mean(preds):.4f}")